#### 讀取需要的函式庫

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from func import *
import jieba
import re
import os

#### 讀取資料夾中的資料

In [2]:
path = '../../107-1PythonSampleCode/week_5/task_5/*.txt'
title, data = readdir(path, 0.6)

Unknown encode file :
../../107-1PythonSampleCode/week_5/task_5/2015-4.txt
../../107-1PythonSampleCode/week_5/task_5/2015-all.txt

Number of unknown encoding files : 2
Number of read files : 4
Number of files : 6


#### 顯示部份資料內容

In [3]:
for i in range(len(data)):
    print('document :', title[i])
    print(data[i][:50], '...', data[i][-50:-1], sep='', end='\n\n')

document : 2012-4
股票代碼:兆利科技工業股份有限公司及其子公司合併財務報表民國一○一年及一○○年十二月三十一日(內附會...及一○○年度其收入占損益表之收入金額%以上之客戶明細如下:年度年度客戶甲$,,客戶乙,,$,,,~

document : 2014-4
壹、致股束報告書回顧-O三年，由於全球經濟景氣復甦緩慢，市場需求表現平淡，依DigitimesRes...訟當事人及截至年報刊印日止之處理情形:無。（十三）其他重要風險及因應措泥:無。七、其他重要事項:無

document : 2016-4
壹、致股東報告書回顧一O五年,由於全球筆電品牌廠面臨關鍵零組件缺貨及漲價壓力,抑制筆記型電腦整體銷售...十六條第三項第二款所定對股東權益或證券價格有重大影響之事項:無。--_______________

document : 2016-all
一???五年度年報中華民國一???六年六月六日刊印本年報內容及本公司相關資訊可至下列網址查詢公開資訊...十六條第三項第二款所定對股東權益或證券價格有重大影響之事項:無。--_______________



#### 新增詞彙至自定義詞庫，並匯入該詞庫
* savetofile(path, list_like_data)
* example for list_like_data:
  * [['來台']] : 完整 '來台' 字串被存入字典中
  * ['來台'] :　'來台' 視為獨立字元，存入 '來' '台' 於字典中 

In [4]:
savetofile('./dict.txt', [['來台']])  # 完整字串　'來台'　被存入字典中
jieba.load_userdict('./dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.063 seconds.
Prefix dict has been built succesfully.


#### 新增詞彙至自定義停止詞彙中

In [5]:
stopwords = ['一','二','三','四','五','六','用','淨','利','值','但','中','可','認','我','無','彙',
             '列','將','對','之','率','已','由','佔','本','日','月','年','另','故','於','較',
             '請','再','經','也','在','依','到','等','或','認列','為','按','比','仍','係','後','及',
             '與','以','時','該','的','並','更','註','例如','可能','由於','因此','附註','部分','事項',
             '各位','晶華','品牌','集團','幅度','資產','新','台幣','營業','受到','提供','使用','本年',
             '說明','現金','成本','營業外','收入','結轉','分攤','其他','影響','帳金額','本會','計師',
             '之會計','報表','會計師','財務','詳合','結果','預測','如下','之一','附註','附','註','其中',
             '併','合','因為','查核','報告','性事','增加','會計','相對','仟元','金額','權益','能力',
             '利益','股東','合理性','假設','無形','上','開關鍵','模型','支出','控制','計算','資金',
             '評估','報告','因應','仟元','金額','權益','假設','無形','上','模型','支出','控制','計算',
             '資金','評估','人次','重大','資訊','總額','民國','估計','','一副','公司','年度','包括',
             '正確性','正','確性','減損','共為','預估','預算','維持','公開','整體','表示','特性','市場',
             '科目','發生','年本','重要','判斷','主要','流量','意見','關鍵','同期','評價','正確','地區',
             '總資產','執行','程序','測試','稅後淨利','股份有限公司']
puncs_Full = '！？｡＂＃＄％＆＇（）()＊＋，－／：；＜＝＞＠［＼］$＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.。'
puncs_Half = "!?.#$%&'()*+,-/:;<=>@[]^_{}|~\\\""
savetofile('./stopword.txt', [stopwords])

In [6]:
stopword = list(map(str.strip, open('./stopword.txt', 'r+').readlines()))
stopword[0:10]

['一', '二', '三', '四', '五', '六', '用', '淨', '利', '值']

#### 利用 jieba 對文件進行斷字，並統計字詞頻率，顯示前十名資料

In [7]:
wordDictionary = []
for i in range(len(data)):
    seg_list = jieba.cut(data[i], cut_all=False)
    seg_list = list(seg_list)
    wordDict = {}
    for w in seg_list:
        if (w not in stopword) and (w not in puncs_Full) and (w not in puncs_Half):
            if w in wordDict:
                wordDict[w] = wordDict[w] + 1
            else:
                wordDict[w] = 1
    wordDictionary.append(wordDict)
    print('Number of word in document ({0}) : '.format(title[i]), len(wordDict))
    sort = sorted(wordDict.items(), key=lambda x: x[1], reverse = True)
    print(sort[0:10])
    data[i] = ' '.join(jieba.cut(data[i], cut_all=False))

Number of word in document (2012-4) :  1985
[('○', 189), ('.%', 119), ('有限公司', 107), ('兆利', 99), ('科技', 97), ('USD', 87), ('所得', 62), ('交易', 61), ('千元', 57), ('其', 57)]
Number of word in document (2014-4) :  3298
[('損益', 202), ('金融', 181), ('負債', 152), ('準則', 132), ('衡量', 124), ('價值', 121), ('所得', 110), ('年月日', 108), ('風險', 101), ('公允', 101)]
Number of word in document (2016-4) :  5739
[('有限公司', 343), ('董事', 254), ('○', 245), ('科技', 220), ('金融', 196), ('衡量', 192), ('.%', 191), ('________________', 183), ('損益', 180), ('價值', 170)]
Number of word in document (2016-all) :  5907
[('有限公司', 348), ('β', 274), ('董事', 263), ('○', 253), ('科技', 224), ('損益', 210), ('________________', 204), ('衡量', 200), ('負債', 200), ('金融', 199)]


#### 顯示各文件低於閥值 (limit) 的字詞

In [8]:
limit = 50
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    for ind, value in wordDictionary[0].items():
        if value <= limit:
            print(ind, end = ', ')
    print()

----Document : 2012-4----
關, 兆旺及, 選擇, 流量表, 香港, 星, 原適, 規定, 提列, 算產生, 業, 方式, 基準, 五年, 及其, 並已, 百分之二十, 贅述, 因, 損分別, 公, 費, 稅須, 依該, RMBRMB, 回, 積投資, 將依, 價格, 嗣, 項達, 及營, 採用, 或民國, 回升, 流動, 高, 工紅利, 彈, 價以, 號之規定採, 資本額, 基金, 而言, 總營收, 房屋, 股票交易, 佈, 風險合, 個, 之個, 中華民國, 監督, 消除, 義務, 生費用, 準, 勞, 依當, 存貨淨, 基本, 並於員, 不動產, 纜, 事業, 抵銷, 第五年, 簧, 產生, 說, 應納營, 第函, 貨自, 資產本, 業已, 之淨值, 起依金, 及兆旺, 擬議, 損後有, 現金產生, 外者, 運決, 財務會計, 不受, 九十八, 元, 用人, 稱關, 福建省, 服務年資, 十四, ________________, 已達, 名稱, 比例, 稅負債, 價為, 壞帳, 福清, 增減, 假, 修訂, 依現, 管會, 負有, 加價, 總應付, 易, 電腦, 創造, 權以, 基礎, 外幣, 吸引, 屬感, 相關者, 損時, 上市, 並訂定, 機構, 招利, 稅資產重, 及光, 應依, 不超, 債權, 均, 估列, 豁免, 外部, 證無須, 歸, 規劃, 轉列, 十月, 項目, 減當, 負擔, 六日, 任則, 不影響, 工過, IFRSs, 制淨利及, 剩餘, 際, 工及, 報表民國, 列無形, 限制, 原因, 付, 雙方間, 無須, 未實現, 之稅額, TopHinge, 及費損, 涵蓋, 外費用, 人民, 之存, 收關, 變動時, 已認, 函之規定, 盈餘未, 型基金, 膠, 部門別, 管理, 權加權, 一般, 其業務性, 稅之會, 相關擬, 銷後之數, 完成, 有, 調節表, 狀況, 集中, 活絡, 發金額, 資金必, 例外, 證六字, 施行, 該項, 擬制, 生, 文號, 之國外, 所產生, 第號, 轉投資, 人才, 支付, 地點, 融通, 之有, 依員工, 續後評, 則假設, 商譽, 提撥, 大陸, 星間, 出售, 盈餘加, 請詳, 因無法, 設備款, 建築物, 執行費, 行, 之淨, 就, 抵質, 備抵, 迴, 理由, 

關, 兆旺及, 選擇, 流量表, 香港, 星, 原適, 規定, 提列, 算產生, 業, 方式, 基準, 五年, 及其, 並已, 百分之二十, 贅述, 因, 損分別, 公, 費, 稅須, 依該, RMBRMB, 回, 積投資, 將依, 價格, 嗣, 項達, 及營, 採用, 或民國, 回升, 流動, 高, 工紅利, 彈, 價以, 號之規定採, 資本額, 基金, 而言, 總營收, 房屋, 股票交易, 佈, 風險合, 個, 之個, 中華民國, 監督, 消除, 義務, 生費用, 準, 勞, 依當, 存貨淨, 基本, 並於員, 不動產, 纜, 事業, 抵銷, 第五年, 簧, 產生, 說, 應納營, 第函, 貨自, 資產本, 業已, 之淨值, 起依金, 及兆旺, 擬議, 損後有, 現金產生, 外者, 運決, 財務會計, 不受, 九十八, 元, 用人, 稱關, 福建省, 服務年資, 十四, ________________, 已達, 名稱, 比例, 稅負債, 價為, 壞帳, 福清, 增減, 假, 修訂, 依現, 管會, 負有, 加價, 總應付, 易, 電腦, 創造, 權以, 基礎, 外幣, 吸引, 屬感, 相關者, 損時, 上市, 並訂定, 機構, 招利, 稅資產重, 及光, 應依, 不超, 債權, 均, 估列, 豁免, 外部, 證無須, 歸, 規劃, 轉列, 十月, 項目, 減當, 負擔, 六日, 任則, 不影響, 工過, IFRSs, 制淨利及, 剩餘, 際, 工及, 報表民國, 列無形, 限制, 原因, 付, 雙方間, 無須, 未實現, 之稅額, TopHinge, 及費損, 涵蓋, 外費用, 人民, 之存, 收關, 變動時, 已認, 函之規定, 盈餘未, 型基金, 膠, 部門別, 管理, 權加權, 一般, 其業務性, 稅之會, 相關擬, 銷後之數, 完成, 有, 調節表, 狀況, 集中, 活絡, 發金額, 資金必, 例外, 證六字, 施行, 該項, 擬制, 生, 文號, 之國外, 所產生, 第號, 轉投資, 人才, 支付, 地點, 融通, 之有, 依員工, 續後評, 則假設, 商譽, 提撥, 大陸, 星間, 出售, 盈餘加, 請詳, 因無法, 設備款, 建築物, 執行費, 行, 之淨, 就, 抵質, 備抵, 迴, 理由, 元件, 之淨利, 銷並, 自, 單獨, 業務, 利息

關, 兆旺及, 選擇, 流量表, 香港, 星, 原適, 規定, 提列, 算產生, 業, 方式, 基準, 五年, 及其, 並已, 百分之二十, 贅述, 因, 損分別, 公, 費, 稅須, 依該, RMBRMB, 回, 積投資, 將依, 價格, 嗣, 項達, 及營, 採用, 或民國, 回升, 流動, 高, 工紅利, 彈, 價以, 號之規定採, 資本額, 基金, 而言, 總營收, 房屋, 股票交易, 佈, 風險合, 個, 之個, 中華民國, 監督, 消除, 義務, 生費用, 準, 勞, 依當, 存貨淨, 基本, 並於員, 不動產, 纜, 事業, 抵銷, 第五年, 簧, 產生, 說, 應納營, 第函, 貨自, 資產本, 業已, 之淨值, 起依金, 及兆旺, 擬議, 損後有, 現金產生, 外者, 運決, 財務會計, 不受, 九十八, 元, 用人, 稱關, 福建省, 服務年資, 十四, ________________, 已達, 名稱, 比例, 稅負債, 價為, 壞帳, 福清, 增減, 假, 修訂, 依現, 管會, 負有, 加價, 總應付, 易, 電腦, 創造, 權以, 基礎, 外幣, 吸引, 屬感, 相關者, 損時, 上市, 並訂定, 機構, 招利, 稅資產重, 及光, 應依, 不超, 債權, 均, 估列, 豁免, 外部, 證無須, 歸, 規劃, 轉列, 十月, 項目, 減當, 負擔, 六日, 任則, 不影響, 工過, IFRSs, 制淨利及, 剩餘, 際, 工及, 報表民國, 列無形, 限制, 原因, 付, 雙方間, 無須, 未實現, 之稅額, TopHinge, 及費損, 涵蓋, 外費用, 人民, 之存, 收關, 變動時, 已認, 函之規定, 盈餘未, 型基金, 膠, 部門別, 管理, 權加權, 一般, 其業務性, 稅之會, 相關擬, 銷後之數, 完成, 有, 調節表, 狀況, 集中, 活絡, 發金額, 資金必, 例外, 證六字, 施行, 該項, 擬制, 生, 文號, 之國外, 所產生, 第號, 轉投資, 人才, 支付, 地點, 融通, 之有, 依員工, 續後評, 則假設, 商譽, 提撥, 大陸, 星間, 出售, 盈餘加, 請詳, 因無法, 設備款, 建築物, 執行費, 行, 之淨, 就, 抵質, 備抵, 迴, 理由, 元件, 之淨利, 銷並, 自, 單獨, 業務, 利息

關, 兆旺及, 選擇, 流量表, 香港, 星, 原適, 規定, 提列, 算產生, 業, 方式, 基準, 五年, 及其, 並已, 百分之二十, 贅述, 因, 損分別, 公, 費, 稅須, 依該, RMBRMB, 回, 積投資, 將依, 價格, 嗣, 項達, 及營, 採用, 或民國, 回升, 流動, 高, 工紅利, 彈, 價以, 號之規定採, 資本額, 基金, 而言, 總營收, 房屋, 股票交易, 佈, 風險合, 個, 之個, 中華民國, 監督, 消除, 義務, 生費用, 準, 勞, 依當, 存貨淨, 基本, 並於員, 不動產, 纜, 事業, 抵銷, 第五年, 簧, 產生, 說, 應納營, 第函, 貨自, 資產本, 業已, 之淨值, 起依金, 及兆旺, 擬議, 損後有, 現金產生, 外者, 運決, 財務會計, 不受, 九十八, 元, 用人, 稱關, 福建省, 服務年資, 十四, ________________, 已達, 名稱, 比例, 稅負債, 價為, 壞帳, 福清, 增減, 假, 修訂, 依現, 管會, 負有, 加價, 總應付, 易, 電腦, 創造, 權以, 基礎, 外幣, 吸引, 屬感, 相關者, 損時, 上市, 並訂定, 機構, 招利, 稅資產重, 及光, 應依, 不超, 債權, 均, 估列, 豁免, 外部, 證無須, 歸, 規劃, 轉列, 十月, 項目, 減當, 負擔, 六日, 任則, 不影響, 工過, IFRSs, 制淨利及, 剩餘, 際, 工及, 報表民國, 列無形, 限制, 原因, 付, 雙方間, 無須, 未實現, 之稅額, TopHinge, 及費損, 涵蓋, 外費用, 人民, 之存, 收關, 變動時, 已認, 函之規定, 盈餘未, 型基金, 膠, 部門別, 管理, 權加權, 一般, 其業務性, 稅之會, 相關擬, 銷後之數, 完成, 有, 調節表, 狀況, 集中, 活絡, 發金額, 資金必, 例外, 證六字, 施行, 該項, 擬制, 生, 文號, 之國外, 所產生, 第號, 轉投資, 人才, 支付, 地點, 融通, 之有, 依員工, 續後評, 則假設, 商譽, 提撥, 大陸, 星間, 出售, 盈餘加, 請詳, 因無法, 設備款, 建築物, 執行費, 行, 之淨, 就, 抵質, 備抵, 迴, 理由, 元件, 之淨利, 銷並, 自, 單獨, 業務, 利息

#### 利用 wordDictionary 移除只出現一次的字詞，並顯示部份結果

In [9]:
for i in range(len(data)):
    for key, value in wordDictionary[0].items():
        if value <= limit:
            data[i] = data[i].replace(key, '')
    print('document :', title[i])
    print(data[i][:50], '...', data[i][-50:-1], sep='', end='\n\n')

document : 2012-4
票  : 利 科技  份 司  子司 合 併   一 ○  及 一 ○ ○ 年 月 三 (   師 ...入   之 入 額 %  之  細  : 年 年   $ , ,   , , $ , , , ~ 

document : 2014-4
壹 、 致 束   顧 - O  ， 由 於 全球 景氣  甦 緩慢 ， 市場  現 平淡 ， 依 ... 日 之   : 無 。 （ 三 ） 其 重  及 應 措泥 : 無 。  、 其 重  : 無 

document : 2016-4
壹 、 致 東  顧 一 O  , 由 於 全球 筆電 牌 廠面 臨鍵  缺及 漲價 壓力 , 抑制...  年 刊印 日 , 發 本法  條     對 東  或    重 影響 之 : 無 。 - -

document : 2016-all
一 ? ? ? 五 年 年  一 ? ? ? 六年   刊印 本年  及 本 司 相 訊 可  網址...  年 刊印 日 , 發 本法  條     對 東  或    重 影響 之 : 無 。 - -



#### 移除所有標點符號、非中文英文數字之資料

In [10]:
for i in range(len(data)):
    data[i] = re.sub('[^\u4e00-\u9fff A-Za-z0-9]+', '', data[i])

#### 印出部份斷字結果

In [11]:
for i in range(len(data)):
    print('document :', title[i])
    print(data[i][:20], ' ... ', data[i][-20:-1], sep='', end='\n\n')

document : 2012-4
票   利 科技  份 司  子司 合  ...   年 年              

document : 2014-4
壹  致 束   顧  O   由 於  ...  應 措泥  無    其 重   無

document : 2016-4
壹  致 東  顧 一 O   由 於  ...   或    重 影響 之  無   

document : 2016-all
一    五 年 年  一    六年  ...   或    重 影響 之  無   



#### 初始化 TFIDF 物件

In [12]:
tfidf_vectorizer = TfidfVectorizer(min_df=0.3, stop_words = stopword)
#tfidf_vectorizer = CountVectorizer(stop_words = stopword)

#### 顯示初始化時輸入的停止詞彙 

In [13]:
print(tfidf_vectorizer.get_stop_words())

frozenset({'', '部分', '無形', '成本', '判斷', '該', '能力', '列', '資金', '會計師', '為', '按', '科目', '對', '無', '佔', '集團', '維持', '如下', '年度', '帳金額', '比', '二', '與', '計算', '年', '本會', '稅後淨利', '台幣', '會計', '正確', '流量', '利', '五', '品牌', '金額', '各位', '權益', '請', '重要', '依', '在', '支出', '開關鍵', '之', '預算', '意見', '表示', '也', '併', '資訊', '詳合', '程序', '淨', '由', '市場', '預估', '係', '再', '或', '公開', '正', '結果', '共為', '收入', '例如', '營業', '以', '特性', '民國', '較', '三', '彙', '分攤', '查核', '財務', '新', '另', '事項', '六', '後', '用', '測試', '性事', '提供', '及', '因為', '年本', '經', '執行', '相對', '關鍵', '計師', '股東', '時', '率', '總資產', '總額', '資產', '利益', '包括', '更', '合', '評價', '並', '其中', '等', '認', '合理性', '使用', '現金', '假設', '仟元', '重大', '仍', '本年', '故', '受到', '其他', '公司', '因應', '說明', '預測', '確性', '之會計', '之一', '四', '報表', '已', '估計', '減損', '中', '但', '一', '於', '我', '營業外', '月', '人次', '註', '結轉', '值', '影響', '本', '的', '將', '股份有限公司', '正確性', '幅度', '因此', '上', '由於', '控制', '一副', '整體', '可能', '增加', '認列', '附註', '報告', '地區', '可', '模型', '附', '評估', '同期', '到', '晶華', '主要', '發生', '日'})


#### 計算 TFIDF 詞頻矩陣

In [14]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data)

#### 顯示詞頻矩陣大小

In [15]:
tfidf_matrix.shape

(4, 1721)

#### 顯示詞頻矩陣中的字詞

In [16]:
words = tfidf_vectorizer.get_feature_names()
words

['aio',
 'aiopc',
 'allinonepc',
 'amd',
 'androidapp',
 'ansys',
 'apple',
 'autolockbk',
 'bcs',
 'bit',
 'bk',
 'black',
 'ca',
 'cb',
 'cc',
 'ces',
 'cg',
 'chianfuhenterpriseco',
 'chrome',
 'chromebook',
 'co',
 'com',
 'consumerdisplays',
 'corei',
 'cpu',
 'cq',
 'dell',
 'digitimes',
 'digitimesresearch',
 'dz',
 'dzdz',
 'dzdzdz',
 'dzdzdzdz',
 'eicc',
 'envycurvedall',
 'eon',
 'eps',
 'fax',
 'fcscstmu',
 'fhd',
 'fhdips',
 'fhdtn',
 'fibertohome',
 'forward',
 'fttb',
 'ftth',
 'fttx',
 'gb',
 'gbddr',
 'gbram',
 'gepon',
 'gpon',
 'great',
 'hd',
 'hdgraphics',
 'hdtn',
 'hinge',
 'hingeholdingsltd',
 'hingetradingltd',
 'holdingsltd',
 'hp',
 'hs',
 'hspm',
 'http',
 'hugearisenind',
 'idc',
 'in',
 'inapprecicationforrecognitionofoutstandingbusinesspartnershipbetweenhewlettpackardandjarllytecco',
 'internet',
 'inyiroad',
 'iso',
 'jarlly',
 'jarllyholdingltd',
 'jumbotechnologylimited',
 'kpmg',
 'lavishinvestmentdevelopmentltd',
 'lcd',
 'lcdmonitor',
 'lcdpc',
 'lcd

In [17]:
for i in range(len(data)):
    print('----Document : {0}----'.format(title[i]))
    for j in range(len(words)):
        if tfidf_matrix[i,j] > 0.2:
              print(words[j], tfidf_matrix[i,j])

----Document : 2012-4----
usd 0.5558818768291514
上海 0.3514195773057853
子司 0.35780902416589055
科技 0.6261657922903084
----Document : 2014-4----
任何 0.2513099849490799
子司 0.44828155603258524
科技 0.3735679633604877
----Document : 2016-4----
子司 0.3204311983788158
監察 0.21616390366824872
科技 0.564569254286485
----Document : 2016-all----
dz 0.22725738999412126
子司 0.30083867577999296
監察 0.2256290068349947
科技 0.53116828692405


In [18]:
dist = cosine_similarity(tfidf_matrix)
print(dist)

[[1.         0.48417944 0.55443932 0.52291677]
 [0.48417944 1.         0.74367711 0.71258199]
 [0.55443932 0.74367711 1.         0.97493999]
 [0.52291677 0.71258199 0.97493999 1.        ]]
